In [1]:
import torch
import pandas as pd
import time
from torch.utils.data import Dataset, TensorDataset
from transformers import BertTokenizer
from torch.nn.utils.rnn import pad_sequence

import torch.nn as nn
from transformers import BertTokenizer,BertModel
from transformers import BertForSequenceClassification

from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients

import transformers
print(transformers.__version__)

4.39.3


In [2]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
# device = torch.device( "cpu")

In [3]:
MAX_LEN = 512
MAX_EPOCHS = 10
BATCH_SIZE = 8
LEARNING_RATE = 1e-5
NUM_CLASSES = 2
WEIGTH_DECAY = 1e-3

In [4]:
def retrieve_train_data():
    train_param = pd.read_json("D:/LTX/code_comment_inconsistency_detection/data/param/train.json")
    train_return = pd.read_json("D:/LTX/code_comment_inconsistency_detection/data/return/train.json")
    train_summary = pd.read_json("D:/LTX/code_comment_inconsistency_detection/data/summary/train.json")
    train_df = pd.concat([train_summary,train_param, train_return], axis=0)
    train_df = train_df.reset_index(drop=True)
    return train_df
def retrieve_valid_data():
    valid_param = pd.read_json("D:/LTX/code_comment_inconsistency_detection/data/param/valid.json")
    valid_return = pd.read_json("D:/LTX/code_comment_inconsistency_detection/data/return/valid.json")
    valid_summary = pd.read_json("D:/LTX/code_comment_inconsistency_detection/data/summary/valid.json")
    valid_df = pd.concat([valid_summary,valid_param, valid_return ], axis=0)
    valid_df = valid_df.reset_index(drop=True)
    return valid_df
def retrieve_test_data():
    test_param = pd.read_json("D:/LTX/code_comment_inconsistency_detection/data/param/test.json")
    test_return = pd.read_json("D:/LTX/code_comment_inconsistency_detection/data/return/test.json")
    test_summary = pd.read_json("D:/LTX/code_comment_inconsistency_detection/data/summary/test.json")
    test_df = pd.concat([test_summary,test_param, test_return], axis=0)
    test_df = test_df.reset_index(drop=True)
    return test_df

In [5]:
train_df = retrieve_train_data()
valid_df = retrieve_valid_data()
test_df = retrieve_test_data()

In [6]:
import javalang
# 用这个，这个对了
# 层次遍历获得层数
def get_blocks_improved_BFS_2(root):
    queue = []
    queue.append(root)
    level = 0
    while queue:
        level = level + 1
        for i in range(len(queue)):
            node = queue.pop(0)
            if hasattr(node, 'children'):
                for child in node.children:
                    queue.append(child)
            else:
                try:
                    len_node = len(node)
                    for i in range(len_node):
                        if hasattr(node[i], 'children'):
                            for child in node[i].children:
                                 queue.append(child)
                except Exception as e:
                    continue
    return level

In [7]:
# 使用javalang解析数据，将AST的层数变为dataframe中的一列，返回新的dataframe

def get_new_df(df):

    error_index = []
    asts = []
    levels = []
    for i in range(len(df)):
        tokens_t = javalang.tokenizer.tokenize(df.loc[i]['new_code_raw'])
        try:
            parser = javalang.parse.Parser(tokens_t)
        except Exception as e:
            error_index.append(i)
            continue
        asts.append(parser)  

    for i in range(len(asts)):
        try:
            # 尝试解析Java代码
            test = asts[i].parse_member_declaration()
            depth = get_blocks_improved_BFS_2(test)
            levels.append(depth)       
        except javalang.parser.JavaSyntaxError:
            # 如果出现JavaSyntaxError，打印错误并跳过
            error_index.append(i)
            print(f"解析错误索引为ast[{i}]，跳过有问题的代码")
            continue 
            
    for i in error_index:
        df.drop(i, inplace=True)
    df = df.reset_index(drop=True)
    
    df['AST_level'] = levels
    
    return df
test_df = get_new_df(test_df)

In [8]:
def get_lines_count(df):
    line_counts = []
    for i in range(len(df)):
        string = df.loc[i]['new_code_raw']
        line_count = len(string.split('\n'))
        line_counts.append(line_count)
    df['line_counts'] = line_counts
    return df
test_df = get_lines_count(test_df)
test_df.head()

,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,new_code_raw,new_code_subtokens,span_diff_code_subtokens,token_diff_code_subtokens,AST_level,line_counts
0,apache_calcite-896-FirstSentence-0,1,Summary,Creates elastic node as single member of a clu...,"[creates, elastic, node, as, single, member, o...",Creates an instance with existing settings,"[creates, an, instance, with, existing, settings]","[<REPLACE_OLD>, elastic, node, as, single, mem...",public static EmbeddedElasticsearchNode crea...,"[public, static, embedded, elasticsearch, node...",private static EmbeddedElasticsearchNode cre...,"[private, static, embedded, elasticsearch, nod...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...",8,6
1,hibernate_hibernate_orm-1601-FirstSentence-0,0,Summary,Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",[],\tpublic boolean areInsertionsOrDeletionsQueue...,"[public, boolean, are, insertions, or, deletio...",\tpublic boolean areInsertionsOrDeletionsQueue...,"[public, boolean, are, insertions, or, deletio...","[<KEEP>, public, boolean, are, insertions, or,...","[<KEEP>, public, <KEEP>, boolean, <KEEP>, are,...",7,4
2,apache_giraph-33-Associations-FirstSentence,1,Summary,Marshal the aggregator values of to a JSONArra...,"[marshal, the, aggregator, values, of, to, a, ...",Marshal the aggregator values of the worker to...,"[marshal, the, aggregator, values, of, the, wo...","[<INSERT_OLD_KEEP_BEFORE>, of, <INSERT_NEW_KEE...",private JSONArray marshalAggregatorValues(lo...,"[private, jsonarray, marshal, aggregator, valu...",private byte[] marshalAggregatorValues(long ...,"[private, byte, [, ], marshal, aggregator, val...","[<KEEP>, private, <KEEP_END>, <REPLACE_OLD>, j...","[<KEEP>, private, <REPLACE_OLD>, jsonarray, <R...",13,28
3,apache_calcite-677-FirstSentence-0,0,Summary,Returns a list of the types of the fields in a...,"[returns, a, list, of, the, types, of, the, fi...",Returns a list of the types of the fields in a...,"[returns, a, list, of, the, types, of, the, fi...",[],public static List<RelDataType> getFieldType...,"[public, static, list, <, rel, data, type, >, ...",public static List<RelDataType> getFieldType...,"[public, static, list, <, rel, data, type, >, ...","[<KEEP>, public, static, list, <, rel, data, t...","[<KEEP>, public, <KEEP>, static, <KEEP>, list,...",5,4
4,apache_calcite-315-Associations-FirstSentence,1,Summary,Create an instance based on current maven prof...,"[create, an, instance, based, on, current, mav...",Creates an instance based on current maven pro...,"[creates, an, instance, based, on, current, ma...","[<REPLACE_OLD>, create, <REPLACE_NEW>, creates...",static MongoDatabaseRule create() {\n fin...,"[static, mongo, database, rule, create, (, ), ...",static MongoDatabasePolicy create() {\n f...,"[static, mongo, database, policy, create, (, )...","[<KEEP>, static, mongo, database, <KEEP_END>, ...","[<KEEP>, static, <KEEP>, mongo, <KEEP>, databa...",10,16


In [9]:
def format_data(df):
    old_code_raw = df['new_code_raw']
    old_code_raw = old_code_raw.values
    old_code_raw = [str(ele) for ele in old_code_raw]
       
    multi_line_old_code = []
    for i in range(len(old_code_raw)):
        multi_line_test = old_code_raw[i].replace('\n', ' ')   # 去掉\n
        multi_line_test = ' '.join(multi_line_test.split())    # 把多余空格变成一个空格
        multi_line_old_code.append(multi_line_test) 
     
    old_comment_raw = df['old_comment_raw']
    old_comment_raw = old_comment_raw.values
    old_comment_raw = [str(ele) for ele in old_comment_raw]
    multi_line_old_comment = []
    for i in range(len(old_comment_raw)):
        multi_line_test = ' '.join(old_comment_raw[i].split())    # 把多余空格变成一个空格
        multi_line_old_comment.append(multi_line_test)  
    
    df['new_code_raw'] = multi_line_old_code
    df['old_comment_raw'] = multi_line_old_comment
    
    return df

In [10]:
import re

def get_token_len(df):
    old_comment_token_num = []
    new_code_token_num = []
    old_comment_tokens = []
    new_code_tokens = []

    for line in range(len(df)):
        code = df.loc[line]['new_code_raw']
        comment = df.loc[line]['old_comment_raw']
        code_tokens = re.findall(r'\b\w+\b|\(|\)|\[|\]|\{|\}|\,|\;|\:|\'|\"|\\|\/|\||\?|\!|\@|\#|\$|\%|\^|\&|\*|\-|\+|\=\<\>', code)
        comment_tokens = re.findall(r'\b\w+\b|\(|\)|\[|\]|\{|\}|\,|\;|\:|\'|\"|\\|\/|\||\?|\!|\@|\#|\$|\%|\^|\&|\*|\-|\+|\=\<\>', comment)
        new_code_tokens.append(code_tokens)
        old_comment_tokens.append(comment_tokens)
        new_code_token_num.append(len(code_tokens))
        old_comment_token_num.append(len(comment_tokens))
    df['new_code_tokens'] = new_code_tokens
    df['new_code_token_num'] = new_code_token_num
    df['old_comment_tokens'] = old_comment_tokens
    df['old_comment_token_num'] = old_comment_token_num
    return df

test_df_clean = get_token_len(test_df)

In [11]:
test_df_clean = format_data(test_df_clean)
test_df_clean.head() 

,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,new_code_raw,new_code_subtokens,span_diff_code_subtokens,token_diff_code_subtokens,AST_level,line_counts,new_code_tokens,new_code_token_num,old_comment_tokens,old_comment_token_num
0,apache_calcite-896-FirstSentence-0,1,Summary,Creates elastic node as single member of a clu...,"[creates, elastic, node, as, single, member, o...",Creates an instance with existing settings,"[creates, an, instance, with, existing, settings]","[<REPLACE_OLD>, elastic, node, as, single, mem...",public static EmbeddedElasticsearchNode crea...,"[public, static, embedded, elasticsearch, node...",private static EmbeddedElasticsearchNode creat...,"[private, static, embedded, elasticsearch, nod...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...",8,6,"[private, static, EmbeddedElasticsearchNode, c...",48,"[Creates, elastic, node, as, single, member, o...",9
1,hibernate_hibernate_orm-1601-FirstSentence-0,0,Summary,Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",[],\tpublic boolean areInsertionsOrDeletionsQueue...,"[public, boolean, are, insertions, or, deletio...",public boolean areInsertionsOrDeletionsQueued(...,"[public, boolean, are, insertions, or, deletio...","[<KEEP>, public, boolean, are, insertions, or,...","[<KEEP>, public, <KEEP>, boolean, <KEEP>, are,...",7,4,"[public, boolean, areInsertionsOrDeletionsQueu...",30,"[Check, whether, any, insertion, or, deletion,...",10
2,apache_giraph-33-Associations-FirstSentence,1,Summary,Marshal the aggregator values of to a JSONArra...,"[marshal, the, aggregator, values, of, to, a, ...",Marshal the aggregator values of the worker to...,"[marshal, the, aggregator, values, of, the, wo...","[<INSERT_OLD_KEEP_BEFORE>, of, <INSERT_NEW_KEE...",private JSONArray marshalAggregatorValues(lo...,"[private, jsonarray, marshal, aggregator, valu...",private byte[] marshalAggregatorValues(long su...,"[private, byte, [, ], marshal, aggregator, val...","[<KEEP>, private, <KEEP_END>, <REPLACE_OLD>, j...","[<KEEP>, private, <REPLACE_OLD>, jsonarray, <R...",13,28,"[private, byte, [, ], marshalAggregatorValues,...",153,"[Marshal, the, aggregator, values, of, to, a, ...",15
3,apache_calcite-677-FirstSentence-0,0,Summary,Returns a list of the types of the fields in a...,"[returns, a, list, of, the, types, of, the, fi...",Returns a list of the types of the fields in a...,"[returns, a, list, of, the, types, of, the, fi...",[],public static List<RelDataType> getFieldType...,"[public, static, list, <, rel, data, type, >, ...",public static List<RelDataType> getFieldTypeLi...,"[public, static, list, <, rel, data, type, >, ...","[<KEEP>, public, static, list, <, rel, data, t...","[<KEEP>, public, <KEEP>, static, <KEEP>, list,...",5,4,"[public, static, List, RelDataType, getFieldTy...",24,"[Returns, a, list, of, the, types, of, the, fi...",14
4,apache_calcite-315-Associations-FirstSentence,1,Summary,Create an instance based on current maven prof...,"[create, an, instance, based, on, current, mav...",Creates an instance based on current maven pro...,"[creates, an, instance, based, on, current, ma...","[<REPLACE_OLD>, create, <REPLACE_NEW>, creates...",static MongoDatabaseRule create() {\n fin...,"[static, mongo, database, rule, create, (, ), ...",static MongoDatabasePolicy create() { final Mo...,"[static, mongo, database, policy, create, (, )...","[<KEEP>, static, mongo, database, <KEEP_END>, ...","[<KEEP>, static, <KEEP>, mongo, <KEEP>, databa...",10,16,"[static, MongoDatabasePolicy, create, (, ), {,...",98,"[Create, an, instance, based, on, current, mav...",15


In [12]:
# model = torch.load('save_GCBmodel.pt',map_location=torch.device('cuda:0'))
model = torch.load('save_bertmodel.pt',map_location=torch.device('cuda:2'))

model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [13]:
device

device(type='cuda', index=2)

In [14]:
tokenizer = BertTokenizer.from_pretrained('D:/LTX/models--bert-base-uncased')

In [15]:
class CocoDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.tokenizer = BertTokenizer.from_pretrained('D:/LTX/models--bert-base-uncased', do_lower_case=True)
        self.data = self.load_data(self.df)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

    def load_data(self, df):
        token_ids = []
        mask_ids = []
        seg_ids = []
        labels = []
        
        code_list = df['new_code_raw'].to_list() 
        comment_list = df['old_comment_raw'].to_list()
        label_list = df['label'].to_list()

        for (code, comment, label) in zip(code_list, comment_list, label_list):
            code_id = self.tokenizer.encode(code, add_special_tokens=False, truncation=True, max_length=512)
            comment_id = self.tokenizer.encode(comment, add_special_tokens=False, truncation=True, max_length=512)

            # want [CLS] comment tokens [SEP] code tokens [SEP]
            pair_token_ids = [self.tokenizer.cls_token_id] + comment_id + [self.tokenizer.sep_token_id] + code_id + [self.tokenizer.sep_token_id]
            pair_token_ids = self.truncate(pair_token_ids)
            code_len = len(code_id)
            comment_len = len(comment_id)
            
            attention_mask_ids = torch.tensor([1] * (code_len + comment_len + 3)) # mask padded values
            segment_ids = torch.tensor([0] * (comment_len + 2) + [1] * (code_len + 1)) # sentence 0 (comment) and sentence 1 (code)
            
            attention_mask_ids = self.truncate(attention_mask_ids)
            segment_ids = self.truncate(segment_ids)
            
            token_ids.append(torch.tensor(pair_token_ids))
            mask_ids.append(attention_mask_ids)
            seg_ids.append(segment_ids)
            labels.append(label)
            
        token_ids = pad_sequence(token_ids, batch_first=True)
        mask_ids = pad_sequence(mask_ids, batch_first=True)
        seg_ids = pad_sequence(seg_ids, batch_first=True)
        labels = torch.tensor(labels)
        
        dataset = TensorDataset(token_ids, mask_ids, seg_ids, labels)
        return dataset

    def truncate(self, ids):
        return ids[:MAX_LEN] if len(ids) > MAX_LEN else ids

In [16]:
from torch.utils.data import DataLoader

test_data = CocoDataset(test_df_clean)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=True)

In [17]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def compute_metrics(predicted_labels, gold_labels):
    predicted_labels = [label.item() for label in predicted_labels]
    gold_labels = [label.item() for label in gold_labels]

    assert len(predicted_labels) == len(gold_labels)

    precision = precision_score(gold_labels, predicted_labels, zero_division=0)
    recall = recall_score(gold_labels, predicted_labels, zero_division=0)
    f1 = f1_score(gold_labels, predicted_labels, zero_division=0)
    accuracy = accuracy_score(gold_labels, predicted_labels) 

    return precision, recall, f1, accuracy

In [18]:
def test(model, test_loader, device):
    model.eval()
    test_loss = 0.0
    predictions = []
    gold_labels = []

    with torch.no_grad():
        for batch_idx, (sequence, attention_masks, token_type_ids, labels) in enumerate(test_loader):
            sequence = sequence.to(device)
            attention_masks = attention_masks.to(device)
            token_type_ids = token_type_ids.to(device)
            labels = labels.to(device)

            outputs = model(sequence, attention_mask=attention_masks, token_type_ids=token_type_ids, labels=labels)
            loss = outputs.loss
            prediction = outputs.logits

            test_loss += loss.item()
            prediction = torch.argmax(prediction, dim=-1)

            predictions.extend(prediction)
            gold_labels.extend(labels)
        
    test_loss = test_loss / len(test_loader)
    test_precision, test_recall, test_f1, test_acc = compute_metrics(predictions, gold_labels)

    print(f"test_loss: {test_loss:.3f} test_precision: {test_precision:.3f} test_recall: {test_recall:.3f} test_f1: {test_f1:.3f} test_acc: {test_acc:.3f}")
    return predictions

In [19]:
predictions = test(model, test_loader, device)

test_loss: 2.715 test_precision: 0.733 test_recall: 0.701 test_f1: 0.717 test_acc: 0.723


In [20]:
test_df_clean['pre_label'] = [x.item() for x in predictions]

In [21]:
test_df_clean.head()

,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,...,new_code_subtokens,span_diff_code_subtokens,token_diff_code_subtokens,AST_level,line_counts,new_code_tokens,new_code_token_num,old_comment_tokens,old_comment_token_num,pre_label
0,apache_calcite-896-FirstSentence-0,1,Summary,Creates elastic node as single member of a clu...,"[creates, elastic, node, as, single, member, o...",Creates an instance with existing settings,"[creates, an, instance, with, existing, settings]","[<REPLACE_OLD>, elastic, node, as, single, mem...",public static EmbeddedElasticsearchNode crea...,"[public, static, embedded, elasticsearch, node...",...,"[private, static, embedded, elasticsearch, nod...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...",8,6,"[private, static, EmbeddedElasticsearchNode, c...",48,"[Creates, elastic, node, as, single, member, o...",9,0
1,hibernate_hibernate_orm-1601-FirstSentence-0,0,Summary,Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",[],\tpublic boolean areInsertionsOrDeletionsQueue...,"[public, boolean, are, insertions, or, deletio...",...,"[public, boolean, are, insertions, or, deletio...","[<KEEP>, public, boolean, are, insertions, or,...","[<KEEP>, public, <KEEP>, boolean, <KEEP>, are,...",7,4,"[public, boolean, areInsertionsOrDeletionsQueu...",30,"[Check, whether, any, insertion, or, deletion,...",10,1
2,apache_giraph-33-Associations-FirstSentence,1,Summary,Marshal the aggregator values of to a JSONArra...,"[marshal, the, aggregator, values, of, to, a, ...",Marshal the aggregator values of the worker to...,"[marshal, the, aggregator, values, of, the, wo...","[<INSERT_OLD_KEEP_BEFORE>, of, <INSERT_NEW_KEE...",private JSONArray marshalAggregatorValues(lo...,"[private, jsonarray, marshal, aggregator, valu...",...,"[private, byte, [, ], marshal, aggregator, val...","[<KEEP>, private, <KEEP_END>, <REPLACE_OLD>, j...","[<KEEP>, private, <REPLACE_OLD>, jsonarray, <R...",13,28,"[private, byte, [, ], marshalAggregatorValues,...",153,"[Marshal, the, aggregator, values, of, to, a, ...",15,0
3,apache_calcite-677-FirstSentence-0,0,Summary,Returns a list of the types of the fields in a...,"[returns, a, list, of, the, types, of, the, fi...",Returns a list of the types of the fields in a...,"[returns, a, list, of, the, types, of, the, fi...",[],public static List<RelDataType> getFieldType...,"[public, static, list, <, rel, data, type, >, ...",...,"[public, static, list, <, rel, data, type, >, ...","[<KEEP>, public, static, list, <, rel, data, t...","[<KEEP>, public, <KEEP>, static, <KEEP>, list,...",5,4,"[public, static, List, RelDataType, getFieldTy...",24,"[Returns, a, list, of, the, types, of, the, fi...",14,0
4,apache_calcite-315-Associations-FirstSentence,1,Summary,Create an instance based on current maven prof...,"[create, an, instance, based, on, current, mav...",Creates an instance based on current maven pro...,"[creates, an, instance, based, on, current, ma...","[<REPLACE_OLD>, create, <REPLACE_NEW>, creates...",static MongoDatabaseRule create() {\n fin...,"[static, mongo, database, rule, create, (, ), ...",...,"[static, mongo, database, policy, create, (, )...","[<KEEP>, static, mongo, database, <KEEP_END>, ...","[<KEEP>, static, <KEEP>, mongo, <KEEP>, databa...",10,16,"[static, MongoDatabasePolicy, create, (, ), {,...",98,"[Create, an, instance, based, on, current, mav...",15,0


In [22]:
test_df_get_unsame = test_df_clean[test_df_clean['label'] != test_df_clean['pre_label']]
test_df_get_unsame = test_df_get_unsame.reset_index(drop=True)

In [23]:
print(len(test_df_get_unsame))
test_df_get_unsame.head()

2018


,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,...,new_code_subtokens,span_diff_code_subtokens,token_diff_code_subtokens,AST_level,line_counts,new_code_tokens,new_code_token_num,old_comment_tokens,old_comment_token_num,pre_label
0,apache_calcite-896-FirstSentence-0,1,Summary,Creates elastic node as single member of a clu...,"[creates, elastic, node, as, single, member, o...",Creates an instance with existing settings,"[creates, an, instance, with, existing, settings]","[<REPLACE_OLD>, elastic, node, as, single, mem...",public static EmbeddedElasticsearchNode crea...,"[public, static, embedded, elasticsearch, node...",...,"[private, static, embedded, elasticsearch, nod...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...",8,6,"[private, static, EmbeddedElasticsearchNode, c...",48,"[Creates, elastic, node, as, single, member, o...",9,0
1,hibernate_hibernate_orm-1601-FirstSentence-0,0,Summary,Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",[],\tpublic boolean areInsertionsOrDeletionsQueue...,"[public, boolean, are, insertions, or, deletio...",...,"[public, boolean, are, insertions, or, deletio...","[<KEEP>, public, boolean, are, insertions, or,...","[<KEEP>, public, <KEEP>, boolean, <KEEP>, are,...",7,4,"[public, boolean, areInsertionsOrDeletionsQueu...",30,"[Check, whether, any, insertion, or, deletion,...",10,1
2,apache_giraph-33-Associations-FirstSentence,1,Summary,Marshal the aggregator values of to a JSONArra...,"[marshal, the, aggregator, values, of, to, a, ...",Marshal the aggregator values of the worker to...,"[marshal, the, aggregator, values, of, the, wo...","[<INSERT_OLD_KEEP_BEFORE>, of, <INSERT_NEW_KEE...",private JSONArray marshalAggregatorValues(lo...,"[private, jsonarray, marshal, aggregator, valu...",...,"[private, byte, [, ], marshal, aggregator, val...","[<KEEP>, private, <KEEP_END>, <REPLACE_OLD>, j...","[<KEEP>, private, <REPLACE_OLD>, jsonarray, <R...",13,28,"[private, byte, [, ], marshalAggregatorValues,...",153,"[Marshal, the, aggregator, values, of, to, a, ...",15,0
3,apache_calcite-315-Associations-FirstSentence,1,Summary,Create an instance based on current maven prof...,"[create, an, instance, based, on, current, mav...",Creates an instance based on current maven pro...,"[creates, an, instance, based, on, current, ma...","[<REPLACE_OLD>, create, <REPLACE_NEW>, creates...",static MongoDatabaseRule create() {\n fin...,"[static, mongo, database, rule, create, (, ), ...",...,"[static, mongo, database, policy, create, (, )...","[<KEEP>, static, mongo, database, <KEEP_END>, ...","[<KEEP>, static, <KEEP>, mongo, <KEEP>, databa...",10,16,"[static, MongoDatabasePolicy, create, (, ), {,...",98,"[Create, an, instance, based, on, current, mav...",15,0
4,apache_jclouds-356-Associations-FirstSentence,1,Summary,Returns the providers that are bound to the sa...,"[returns, the, providers, that, are, bound, to...",Returns the providers that are bound to the sa...,"[returns, the, providers, that, are, bound, to...","[<REPLACE_OLD>, type, <REPLACE_NEW>, context, ...","public static Iterable<ProviderMetadata<?, ...","[public, static, iterable, <, provider, metada...",...,"[public, static, iterable, <, provider, metada...","[<KEEP>, public, static, iterable, <, provider...","[<KEEP>, public, <KEEP>, static, <KEEP>, itera...",7,7,"[public, static, Iterable, ProviderMetadata, b...",41,"[Returns, the, providers, that, are, bound, to...",21,0


In [24]:
test_df_get_unsame = test_df_get_unsame.reset_index(drop=True)


In [25]:
# label = 1  一致
print(test_df_get_unsame.loc[0]['old_comment_raw'])
print(test_df_get_unsame.loc[0]['new_comment_raw'])
print(test_df_get_unsame.loc[0]['old_code_raw'])
print(test_df_get_unsame.loc[0]['new_code_raw'])

Creates elastic node as single member of a cluster.
Creates an instance with existing settings
  public static EmbeddedElasticsearchNode create() {
    File data = Files.createTempDir();
    data.deleteOnExit();
    File home = Files.createTempDir();
    home.deleteOnExit();

    Settings settings = Settings.builder()
        .put("node.name", "fake-elastic")
        .put("path.home", home.getAbsolutePath())
        .put("path.data", data.getAbsolutePath())
        .put("http.type", "netty4")
        .put("network.host", "localhost")
        .build();

    return create(settings);
  }

private static EmbeddedElasticsearchNode create(Settings settings) { // ensure PainlessPlugin is installed or otherwise scripted fields would not work Node node = new LocalNode(settings, Arrays.asList(Netty4Plugin.class, PainlessPlugin.class)); return new EmbeddedElasticsearchNode(node); }


In [26]:
# label = 0  不一致
print(test_df_get_unsame.loc[1]['old_comment_raw'])
print(test_df_get_unsame.loc[1]['new_comment_raw'])
print(test_df_get_unsame.loc[1]['old_code_raw'])
print(test_df_get_unsame.loc[1]['new_code_raw'])

Check whether any insertion or deletion actions are currently queued.
Check whether any insertion or deletion actions are currently queued.
	public boolean areInsertionsOrDeletionsQueued() {
		return ( insertions.size() > 0 || deletions.size() > 0 );
	}

public boolean areInsertionsOrDeletionsQueued() { return ( insertions.size() > 0 || ! unresolvedInsertions.isEmpty() || deletions.size() > 0 ); }


In [27]:
len(test_df_get_unsame)

2018

In [28]:
selected_columns = ['new_code_raw','old_code_raw','new_comment_raw','old_comment_raw','label', 'pre_label','AST_level','line_counts','new_code_token_num']

test_df_get_unsame_clean = test_df_get_unsame.loc[:, selected_columns]

test_df_get_unsame_clean.head()

,new_code_raw,old_code_raw,new_comment_raw,old_comment_raw,label,pre_label,AST_level,line_counts,new_code_token_num
0,private static EmbeddedElasticsearchNode creat...,public static EmbeddedElasticsearchNode crea...,Creates an instance with existing settings,Creates elastic node as single member of a clu...,1,0,8,6,48
1,public boolean areInsertionsOrDeletionsQueued(...,\tpublic boolean areInsertionsOrDeletionsQueue...,Check whether any insertion or deletion action...,Check whether any insertion or deletion action...,0,1,7,4,30
2,private byte[] marshalAggregatorValues(long su...,private JSONArray marshalAggregatorValues(lo...,Marshal the aggregator values of the worker to...,Marshal the aggregator values of to a JSONArra...,1,0,13,28,153
3,static MongoDatabasePolicy create() { final Mo...,static MongoDatabaseRule create() {\n fin...,Creates an instance based on current maven pro...,Create an instance based on current maven prof...,1,0,10,16,98
4,public static Iterable<ProviderMetadata> bound...,"public static Iterable<ProviderMetadata<?, ...",Returns the providers that are bound to the sa...,Returns the providers that are bound to the sa...,1,0,7,7,41


试着统计分析 分析AST长度分布状况

In [29]:
# 打印出csv，不要txt试试
test_df_get_unsame_clean.to_json('Bert_test_interpration_json.json', orient='records')